In [89]:
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
import requests
import io
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
imputer_num = SimpleImputer(missing_values=np.nan, strategy='mean')
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
imputer_cat = SimpleImputer(strategy="most_frequent")
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from flytekit import task, workflow
from sklearn.preprocessing import LabelEncoder


In [90]:
url = "https://github.com/smadarab/flytelab/raw/main/census.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')),sep=',')
print("df is created",df.columns)
#df.dropna(inplace=True)
#df = df.reset_index()
print(df.columns)
train = df

df is created Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')
Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country',
       'income'],
      dtype='object')


In [91]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from joblib import dump, load


In [92]:
ohe = OneHotEncoder(handle_unknown = 'ignore')
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loos', 'hour-per-week']
cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
log_transform_cols = ['capital-loos', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def one_hot_encode(X):
    print("one_hot_encode")
    ohe = OneHotEncoder(handle_unknown = 'ignore')
    print(X[0].shape)
    ohe.fit(pd.DataFrame(X))
    dump(ohe, 'onehot.joblib') 
    import subprocess
    subprocess.call(["git", "add","."])
    subprocess.call(["git", "commit","-m","kfbjebfe"])
    subprocess.call(["git", "push"])
    y = ohe.transform(pd.DataFrame(X)).toarray()
    print('\n \n',y)
    return y
def label_encode(X):
    print("label_encode")
    df = pd.DataFrame(X)
    le = LabelEncoder()
    df[cols] = df[cols].apply(LabelEncoder().fit_transform)
    df.status = le.fit_transform(X)
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))    
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])


In [93]:
full_pipeline = Pipeline([('steps_', steps_)])
y = train['income'].map({'<=50K': 0, '>50K': 1})
X = full_pipeline.fit_transform(train)

one_hot_encode
(7,)
[my_project d0625ac] kfbjebfe
 1 file changed, 0 insertions(+), 0 deletions(-)
 rewrite projects/my_project/my_project/onehot.joblib (63%)

 
 [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


To https://github.com/smadarab/flytelab
   9753754..d0625ac  my_project -> my_project


In [104]:
age =  50
education_num =  13.0
capital_gain =  0.0
capital_loos = 0.0
hour_per_week = 13.0
workclass = "Self-emp-not-inc"
marital_status = "Married-civ-spouse"
occupation = "Exec-managerial"
relationship = "Husband"
race = "White"
sex = "Male"
native_country ="United-States"
final_weight = 77516
education="Bachelors"

In [105]:
dict_val = {'age':age, 'workclass':workclass, 'final-weight':final_weight, 'education':education, 'education_num':education_num,
       'martial_status':marital_status, 'occupation':occupation, 'relationship':relationship, 'race':race, 'sex':sex,
       'capital-gain':capital_gain, 'capital_loos':capital_loos, 'hour_per_week':hour_per_week, 'native_country':native_country}
       

In [106]:
X_train = pd.DataFrame(dict_val,index=[0])

In [108]:
import os

In [109]:
num_cols = ['age', 'education_num', 'capital-gain',
            'capital_loos', 'hour_per_week']
cat_cols = ['workclass', 
            'martial_status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native_country']
log_transform_cols = ['capital_loos', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    print("in function",X.columns)
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  
def one_hot_encode(X):
    print(X.shape)
    print("current wd",os.getcwd())
    ohe = load('onehot.joblib')
    return ohe.transform(pd.DataFrame(X)).toarray()

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])
full_pipeline = Pipeline([('steps_', steps_)])
X_train = full_pipeline.fit_transform(X_train)

in function Index(['age', 'workclass', 'final-weight', 'education', 'education_num',
       'martial_status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital_loos', 'hour_per_week', 'native_country'],
      dtype='object')
(1, 7)
current wd /Users/apple/flytelab/projects/my_project/my_project


In [110]:
X_train.shape

(1, 93)

In [53]:
url = "https://github.com/smadarab/flytelab/blob/my_project/projects/my_project/my_project/onehot.joblib" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

In [54]:
 ohe = load(download)

OSError: [Errno 63] File name too long: b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en" data-color-mode="auto" data-light-theme="light" data-dark-theme="dark" >\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>\n  <link rel="preconnect" href="https://avatars.githubusercontent.com">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-sD8zc/Dn1lguLc4FdbeVhD2gfxFCTXGr/y+MpJG3oX1vkAyaFf/7BHPtvv1dAHA0KG4bDEW3Ex1EPrzGTMtN2Q==" rel="stylesheet" href="https://github.githubassets.com/assets/light-b03f3373f0e7.css" /><link crossorigin="anonymous" media="all" integrity="sha512-V0872pNNqPTaPQFzdwX9tHIzJvB4F1Foi6VYDciZAMo/qHgBC8

In [55]:
ohe.transform(pd.DataFrame(X)).toarray()

NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [86]:
model = AdaBoostClassifier(n_estimators=300)
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train = np.nan_to_num(X_train)
y_train=np.nan_to_num(y_train)
print("X_train dimensiona",X_train)
model.fit(X_train, y_train)

X_train dimensiona [[ 0.          0.          0.52054795 ...  1.          0.
   0.        ]
 [ 0.          0.          0.32876712 ...  1.          0.
   0.        ]
 [ 0.          0.          0.36986301 ...  1.          0.
   0.        ]
 ...
 [ 0.          0.          0.10958904 ...  1.          0.
   0.        ]
 [ 0.         10.23383392  0.4109589  ...  1.          0.
   0.        ]
 [ 0.          0.          0.1369863  ...  1.          0.
   0.        ]]


AdaBoostClassifier(n_estimators=300)

# Test part

In [87]:
num_cols = ['age', 'education-num', 'capital-gain',
            'capital-loos', 'hour-per-week']
cat_cols = ['workclass', 
            'marital-status', 'occupation', 
            'relationship', 'race', 
            'sex', 'native-country']
log_transform_cols = ['capital-loos', 'capital-gain']    
def get_cat_cols(X):
    return X[cat_cols]
def get_num_cols(X):
    return X[num_cols]
def get_log_transform_cols(X):
    return X[log_transform_cols]
def get_dummies(X):
    print('\n \n',type(X))
    return pd.get_dummies(pd.DataFrame(X))
def one_hot_encode(X):
    print("one_hot_encode")
    ohe = load('filename.joblib') 
    y = ohe.transform(pd.DataFrame(X)).toarray()
    print('\n \n',y)
    return y
def label_encode(X):
    print("label_encode")
    df = pd.DataFrame(X)
    le = LabelEncoder()
    df[cols] = df[cols].apply(LabelEncoder().fit_transform)
    df.status = le.fit_transform(X)
def cat_imputer(X):
    print(X.shape)
    return(imputer_cat.fit_transform(X))
    #return X.apply(lambda col: imputer_cat.fit_transform(col))  

log_transform_pipeline = Pipeline([
('get_log_transform_cols', FunctionTransformer(get_log_transform_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),   
('log_transform', FunctionTransformer(np.log1p))
])

num_cols_pipeline = Pipeline([
('get_num_cols', FunctionTransformer(get_num_cols, validate=False)),
('imputer', SimpleImputer(strategy='mean')),
('min_max_scaler', MinMaxScaler())
])

cat_cols_pipeline = Pipeline([
('get_cat_cols', FunctionTransformer(get_cat_cols, validate=False)),
('imputer', SimpleImputer(strategy="most_frequent")),
#('get_dummies', FunctionTransformer(get_dummies, validate=False))
('one_hot_encode', FunctionTransformer(one_hot_encode, validate=False))    
])       

steps_ = FeatureUnion([
('log_transform', log_transform_pipeline),
('num_cols', num_cols_pipeline),
('cat_cols', cat_cols_pipeline)
])

In [88]:
age =  50
education_num =  13.0
capital_gain =  0.0
capital_loos = 0.0
hour_per_week = 13.0
workclass = "Self-emp-not-inc"
marital_status = "Married-civ-spouse"
occupation = "Exec-managerial"
relationship = "Husband"
race = "White"
sex = "Male"
native_country ="United-States"
final_weight = 77516
education="Bachelors"

In [89]:
dict_val = {'age':age, 'workclass':workclass, 'final-weight':final_weight, 'education':education, 'education-num':education_num,
       'marital-status':marital_status, 'occupation':occupation, 'relationship':relationship, 'race':race, 'sex':sex,
       'capital-gain':capital_gain, 'capital-loos':capital_loos, 'hour-per-week':hour_per_week, 'native-country':native_country}
       

In [90]:
X_train = pd.DataFrame(dict_val,index=[0])

In [91]:
X_train

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country
0,50,Self-emp-not-inc,77516,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States


In [92]:
full_pipeline = Pipeline([('steps_', steps_)])
X = full_pipeline.fit_transform(X_train)

one_hot_encode

 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


/root/flyte_hack/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [94]:
y = model.predict(X)

In [95]:
y

array([0.])